In [ ]:
# 简介

# 部署过程

cd /data/

conda create -n zimage python=3.10
conda activate zimage

#安装diffuser，记住从源码安装，具体原因参见官网(https://github.com/Tongyi-MAI/Z-Image)
git clone https://github.com/huggingface/diffusers.git
cd diffusers
pip install -e .

#安装pytorch
nvcc --version

#根据cuda版本，去https://pytorch.org/get-started/locally/选择对应的参数，拷贝安装指令执行即可。
#因为我本机的cuda版本是12.8，所以安装的指令是：
pip3 install torch torchvision --index-url https://download.pytorch.org/whl/cu128

#安装transformers
pip install transformers

# 安装modelscope，下载模型
pip install modelscope
modelscope download --model Tongyi-MAI/Z-Image-Turbo


# 示例代码

In [ ]:
import torch
from diffusers import ZImagePipeline

# 1. Load the pipeline
# Use bfloat16 for optimal performance on supported GPUs
pipe = ZImagePipeline.from_pretrained(
    "/root/.cache/modelscope/hub/models/Tongyi-MAI/Z-Image-Turbo",
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=False,
)
pipe.to("cuda")

# 官网给的例子
prompt2 = "杰作，最佳质量，超逼真照片，东亚美女，UHD，一张中幅照片，一位面容精致、纯净空灵的美丽年轻女子。她身着专业运动服装，上身是一件剪裁利落的浅灰色速干运动夹克，搭配同色系透气网眼内搭，下身是高弹修身运动长裤勾勒出纤细线条，脚踩轻便缓震运动鞋，整体造型兼具功能性与时尚感。她站在城市滨江步道的开阔草坪上，时值活力清晨时刻。光线清亮而通透，将初升朝阳的金橙色光芒与晨雾未散的清新冷调自然交织。层次化构图以虚化的远处江面波光与近景摇曳的狗尾草为衬，营造出动感与宁静交融的运动氛围，精准传递出服装在动态场景下的舒适包裹感与轻盈呼吸感。"

image = pipe(
    prompt=prompt2,
    height=1920,
    width=1440,
    num_inference_steps=9,  # 越高质量越好，但是推理时间也会增加，建议10-50之间
    guidance_scale=0.0,     # Guidance should be 0 for the Turbo models
    generator=torch.Generator("cuda").manual_seed(42),
).images[0]

image.save("test.png")


# 效果图